In [1]:
# Training process was done by YOLOv2 (AlexeyAB)
# The code is available at https://github.com/AlexeyAB/darknet/tree/47c7af1cea5bbdedf1184963355e6418cb8b1b4f
# Please import this Notebook into 'darknet' folder that you just cloned from the Github

In [3]:
from ctypes import *
import math
import random
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from darknet import *
import json

### Preprocessing: Image Augmentation

In [ ]:
num_of_repetition= {1:int(1.88*13861),
    2:int(2.77*10585),
    3:int(3.70*8497),
    4:int(4.36*7458),
    5:int(4.81*6882),
    6:int(5.98*5727),
    7:int(6.14*5595),
    8:int(6.92*5045),
    9:int(7.58*4659),
    10:int(7.08*4948)}

# true_random from Quantumrandom package
random_img_idx = np.load('./true_random.npy', allow_pickle=True)
random_img_idx = list(random_img_idx)
_temp = [random_img_idx[-1] for i in range(10)]
random_img_idx += _temp

# flatten
_temp = []
for i in random_img_idx:
    for j in i:
        _temp.append(j)

np.random.seed(123)
random_img_idx = _temp
random_img_idx+=random_img_idx

np.random.shuffle(random_img_idx)

num_of_repetition_flatten = []
for k,v in num_of_repetition.items():
    for j in range(v):
        num_of_repetition_flatten.append(k)

np.random.shuffle(num_of_repetition_flatten)


train_dir = './data/SVHN/train/'
filenames = os.listdir(train_dir)
save_dir = './data/SVHN/im_aug/'

train_label_dir = './data/SVHN_labels/'
train_label_listdir = os.listdir(train_label_dir)

class_dir = './data/SVHN/number_images/'
class_listdir = os.listdir(class_dir)    

idx1 = 0
idx2 = 0

while True:
    # Get filename of image that we want to be changed
    filename = train_label_listdir[random_img_idx[idx1]]

    # Get its annotation
    with open(train_label_dir+filename, 'r') as f:
        annotation = f.read()

    annotation = annotation.split('\n')
    annotation = [i.split(' ') for i in annotation]

    # Get its image
    im_filename = filename.replace('.txt', '.png')
    _im = cv2.imread(train_dir+im_filename)
    H, W, _ = _im.shape

    _new_annotation =  []

    while True:
        try:
            _cls_f = class_listdir[idx2]
        except:
            idx2 = 0
            _cls_f = class_listdir[idx2]

        _class = _cls_f.strip('.jpg')
        _class = _class.split('_')[-1]
        _class_im = cv2.imread(class_dir+_cls_f)

        # Stopping criterion
        if len(annotation) == 0:
            temp_ant = '\n'.join([' '.join(i) for i in _new_annotation])

            with open(save_dir+str(idx1)+'.txt', 'w') as g:
                g.write(temp_ant)
            break

        if not int(_class) in num_of_repetition_flatten:
            idx2+=1
        else:
            ant_inf = annotation[0]
            cx = float(ant_inf[1]) * W
            cy = float(ant_inf[2]) * H
            x_ = float(ant_inf[3]) * W
            y_ = float(ant_inf[4]) * H
            
            if not cx:                
                _new_annotation += [annotation.pop(0)]
                _new_annotation[-1][0] = str(_class)
                
            else:
                pop_idx = num_of_repetition_flatten.index(int(_class))
                num_of_repetition_flatten.pop(pop_idx)

                if cx-x_/2 < 0:
                    if ant_inf[0] != '1':
                        _im_num = _im[int(cy-y_/2):int(cy+y_/2), 0:int(cx+x_/2),:]
                        _im_num_H, _im_num_W, _ = _im_num.shape
                        temp = cv2.resize(_class_im, (_im_num_W, _im_num_H))
                        _im[int(cy-y_/2):int(cy+y_/2), 0:int(cx+x_/2),:] = temp

                    else:
                        _new_annotation += [annotation.pop(0)]

                else:
                    _im_num = _im[int(cy-y_/2):int(cy+y_/2), int(cx-x_/2):int(cx+x_/2),:]
                    _im_num_H, _im_num_W, _ = _im_num.shape
                    temp = cv2.resize(_class_im, (_im_num_W, _im_num_H))
                    _im[int(cy-y_/2):int(cy+y_/2), int(cx-x_/2):int(cx+x_/2),:] = temp

                # New annotation
                _new_annotation += [annotation.pop(0)]
                _new_annotation[-1][0] = str(_class)
                idx2+=1

    # save new image
    cv2.imwrite(save_dir+str(idx1)+'.jpg', _im)

    idx1+=1

    if len(num_of_repetition_flatten) == 0:
        break

    if len(num_of_repetition_flatten) % 1000 == 0:
        percentage = (1 - (len(num_of_repetition_flatten))/326291)*100
        print('percentage: {0:.3f}'.format(percentage))

print('Finish!')

### Other codes

In [ ]:
# The code is available at https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # initialize the list of picked indexes	
    pick = []
 
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
 
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
 
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
 
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
 
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")

In [5]:
test_dir = '/home/dominikus/study/deep-learning-visual-recognition/HW3/dataset/SVHN_HW/test'
test_img = os.listdir(test_dir)

# order ascendingly, order matter
test_img = [int(i.strip('.png')) for i in test_img]
test_img.sort()
test_img = [str(i)+'.png' for i in test_img]

weight = '/home/dominikus/darknet/backup/yolov2_last.weights'

In [ ]:
# threshold 0.2, no nms
detection_02_nonms = []
for idx, i in enumerate(test_img):
    # im_path = dir_+i
    im_path = test_dir+'/'+i
    detection_02_nonms.append(performDetect(imagePath=im_path, thresh= 0.2, configPath = "./cfg/yolov2.cfg", weightPath = "./backup/yolov2_last.weights", metaPath= "./data/svhn.data", showImage= False, makeImageOnly = False, initOnly= False))
    if idx % 500 == 0:
        print(idx)

In [6]:
def yolo_pred2hw_submission(_input):
    
    if not _input:
        return {"bbox":None, "score":None, "label":None}
    
    labels = []
    bboxes = []
    scores = []
    
    # if labels == 0, change to 10
    
    for i in _input:
        if i[0] == '0':
            labels.append(10)
        else:
            labels.append(int(i[0]))

        scores.append(i[1])

        y1 = int(i[2][1]-i[2][3]/2)
        x1 = int(i[2][0]-i[2][2]/2)
        y2 = int(i[2][1]+i[2][3]/2)
        x2 = int(i[2][0]+i[2][2]/2)
        bboxes.append((y1,x1,y2,x2))
    
    return {"bbox":bboxes, "score":scores, "label":labels}
    
    # these formats are equivalent:
    # hw submission format (y1, x1, y2, x2)
    # opencv rectangle format (x1,y1), (x2,y2)
    # matplotlib format im[y1:y2, x1:x2, :]
    # NMS, create matrix with format (x1,y1,x2,y2)

"\n[('5',\n  0.7317580580711365,\n  (49.79778289794922,\n   23.197275161743164,\n   19.09353256225586,\n   35.48165512084961))]\n\nplt.imshow(iim[23-17:23+17, 49-9:49+9,:])\nround int aja\n\nchange the annotation into submission format\n"

In [ ]:
submission_02_nonms = [yolo_pred2hw_submission(i) for i in detection_02_nonms]

In [ ]:
with open('submission_02_nonms.json', 'w') as f:
    json.dump(submission_02_nonms, f)

In [ ]:
# threshold 0.3, no nms
detection_03_nonms = []
for idx, i in enumerate(test_img):
    # im_path = dir_+i
    im_path = test_dir+'/'+i
    detection_03_nonms.append(performDetect(imagePath=im_path, thresh= 0.3, configPath = "./cfg/yolov2.cfg", weightPath = "./backup/yolov2_last.weights", metaPath= "./data/svhn.data", showImage= False, makeImageOnly = False, initOnly= False))
    if idx % 500 == 0:
        print(idx)

In [ ]:
submission_03_nonms = [yolo_pred2hw_submission(i) for i in detection_03_nonms]

with open('submission_03_nonms.json', 'w') as f:
    json.dump(submission_03_nonms, f)

In [ ]:
def nms(_input, threshold):
    # input is hw submission format

    if not _input['bbox']:
        return {"bbox":None, "score":None, "label":None}

    bboxes = np.zeros((len(_input['bbox']), 4))

    for idx, i in enumerate(_input['bbox']):
        _temp = np.array([i[1], i[0], i[3], i[2]])
        bboxes[idx,:] = _temp
    
    nms_res = non_max_suppression_fast(bboxes, threshold)
    # return nms_res
    
    new_labels = []
    new_scores = []
    new_bbox = []

    for row in range(len(nms_res)):
        _idx = np.where(np.all(bboxes == nms_res[row,:], axis=1))[0][0]
        new_bbox.append(_input['bbox'][_idx])
        new_labels.append(_input['label'][_idx])
        new_scores.append(_input['score'][_idx])
        
    return {"bbox":new_bbox, "score":new_scores, "label":new_labels}

In [ ]:
# submission_1 : threshold 0.2, no nms
# submission_2 : threshold 0.3, no nms

# threshold 0.2, nms 0.5
submission_3 = [nms(i, 0.5) for i in submission_02_nnms]

# threshold 0.2, nms 0.6
submission_4 = [nms(i, 0.6) for i in submission_02_nnms]

# threshold 0.2, nms 0.7
submission_5 = [nms(i, 0.7) for i in submission_02_nnms]

# threshold 0.3, nms 0.5
submission_6 = [nms(i, 0.5) for i in submission_03_nnms]

# threshold 0.3, nms 0.6
submission_7 = [nms(i, 0.6) for i in submission_03_nnms]

# threshold 0.3, nms 0.7
submission_8 = [nms(i, 0.7) for i in submission_03_nnms]

In [ ]:
with open('submission_3.json', 'w') as f:
    json.dump(submission_3, f)
    
with open('submission_4.json', 'w') as f:
    json.dump(submission_4, f)
    
with open('submission_5.json', 'w') as f:
    json.dump(submission_5, f)
    
with open('submission_6.json', 'w') as f:
    json.dump(submission_6, f)
    
with open('submission_7.json', 'w') as f:
    json.dump(submission_7, f)
    
with open('submission_8.json', 'w') as f:
    json.dump(submission_8, f)